# Excercise


In [58]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/heart.csv")
df.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0


In [59]:
df_dumm = pd.get_dummies(df, drop_first=True)
df_dumm.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,True,True,False,False,True,False,False,False,True
1,49,160,180,0,156,1.0,1,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,0,True,True,False,False,False,True,False,False,True
3,48,138,214,0,108,1.5,1,False,False,False,False,True,False,True,True,False
4,54,150,195,0,122,0.0,0,True,False,True,False,True,False,False,False,True


In [60]:
z_scores = np.abs((df_dumm - df_dumm.mean()) / df_dumm.std())
df_no_outliers = df_dumm[(z_scores < 3).all(axis=1)]
df_no_outliers.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,0,True,True,False,False,True,False,False,False,True
1,49,160,180,0,156,1.0,1,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,0,True,True,False,False,False,True,False,False,True
3,48,138,214,0,108,1.5,1,False,False,False,False,True,False,True,True,False
4,54,150,195,0,122,0.0,0,True,False,True,False,True,False,False,False,True


In [61]:
# split
X = df_no_outliers.drop("HeartDisease", axis="columns")
y = df_no_outliers.HeartDisease
X[:5]

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_M,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_Normal,RestingECG_ST,ExerciseAngina_Y,ST_Slope_Flat,ST_Slope_Up
0,40,140,289,0,172,0.0,True,True,False,False,True,False,False,False,True
1,49,160,180,0,156,1.0,False,False,True,False,True,False,False,True,False
2,37,130,283,0,98,0.0,True,True,False,False,False,True,False,False,True
3,48,138,214,0,108,1.5,False,False,False,False,True,False,True,True,False
4,54,150,195,0,122,0.0,True,False,True,False,True,False,False,False,True


In [62]:
# scale first
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
X_scaled[:5]

array([[0.24489796, 0.57142857, 0.55791506, 0.        , 0.78417266,
        0.33333333, 1.        , 1.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 1.        ],
       [0.42857143, 0.76190476, 0.34749035, 0.        , 0.66906475,
        0.5       , 0.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 0.        , 1.        , 0.        ],
       [0.18367347, 0.47619048, 0.54633205, 0.        , 0.25179856,
        0.33333333, 1.        , 1.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        , 1.        ],
       [0.40816327, 0.55238095, 0.41312741, 0.        , 0.32374101,
        0.58333333, 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 1.        , 1.        , 0.        ],
       [0.53061224, 0.66666667, 0.37644788, 0.        , 0.42446043,
        0.33333333, 1.        , 0.        , 1.        , 0.        ,
        1.        , 0.        , 0.        , 

In [63]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, train_size=0.8)

In [64]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score

scores = cross_val_score(SVC(), X_train, y_train, cv=5)
scores.mean()

0.8551202232717905

In [ ]:
from sklearn.ensemble import BaggingClassifier

svc_model = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0,
)


svc_model.fit(X_train, y_train)
# svc_model.oob_score_

svc_model.score(X_test, y_test)

0.9181286549707602

In [66]:
from sklearn.tree import DecisionTreeClassifier

scores = cross_val_score(DecisionTreeClassifier(), X, y, cv=5)
scores.mean()

0.7680220158238733

In [ ]:
tree_model = BaggingClassifier(
    estimator=SVC(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0,
)


tree_model.fit(X_train, y_train)
# tree_model.oob_score_

tree_model.score(X_test, y_test)

0.9181286549707602

In [68]:
from sklearn.ensemble import RandomForestClassifier

scores = cross_val_score(RandomForestClassifier(), X, y, cv=5)
scores.mean()

0.8336429308565532

In [ ]:
r_for_model = BaggingClassifier(
    estimator=RandomForestClassifier(),
    n_estimators=100,
    max_samples=0.8,
    oob_score=True,
    random_state=0,
)


r_for_model.fit(X_train, y_train)
# r_for_model.oob_score_

r_for_model.score(X_test, y_test)

0.935672514619883